In [1]:
import pandas as pd

In [2]:
turnstile_df = pd.read_csv('stat_turn_m.csv')

In [3]:
turnstile_df.head()

,Unnamed: 0,STATION_LINE,DAILY_ENTRIES
0,433,TIMES SQ 42 ST-1237ACENQRSW,156678.0
1,88,34 ST-PENN STA-123ACE,138335.0
2,299,GRD CNTRL-42 ST-4567S,120732.0
3,86,34 ST-HERALD SQ-BDFMNQRW,106933.0
4,26,14 ST-UNION SQ-456LNQRW,84564.0


In [4]:
linker_df = pd.read_csv('lookups.csv')

In [5]:
linker_df.head()

,TurnstileID,LocatorIDs
0,103 ST-CORONA-7,103rd St - Corona Plaza 7
1,103 ST-1,103rd St 1
2,103 ST-6,103rd St 4-6-6 Express
3,103 ST-BC,103rd St A-B-C
4,104 ST-A,104th St A-S


In [8]:
hosp_dist_df = pd.read_csv('station_hospital_distances_old.csv',index_col=0)
hosp_dist_df.head()

,Station Name,Lines,Lat / Long,Hospital Distance (mi),Hospital ID
0,Astor Pl,4-6-6 Express,"(40.73005400028978, -73.99106999861966)",0.367354,NEW YORK EYE AND EAR INFIRMARY
1,Canal St,4-6-6 Express,"(40.71880300107709, -74.00019299927328)",0.631470,NEW YORK-PRESBYTERIAN/LOWER MANHATTAN HOSPITAL
2,50th St,1-2,"(40.76172799961419, -73.98384899986625)",0.588702,MOUNT SINAI WEST
3,Bergen St,2-3-4,"(40.68086213682956, -73.97499915116808)",0.686953,BROOKLYN HOSPITAL CENTER
4,Pennsylvania Ave,3-4,"(40.66471445143568, -73.89488591154061)",1.134439,BROOKDALE HOSPITAL MEDICAL CENTER


In [9]:
hosp_dist_df['StatID'] = hosp_dist_df['Station Name']+' '+hosp_dist_df['Lines']
hosp_dist_df.head()

,Station Name,Lines,Lat / Long,Hospital Distance (mi),Hospital ID,StatID
0,Astor Pl,4-6-6 Express,"(40.73005400028978, -73.99106999861966)",0.367354,NEW YORK EYE AND EAR INFIRMARY,Astor Pl 4-6-6 Express
1,Canal St,4-6-6 Express,"(40.71880300107709, -74.00019299927328)",0.631470,NEW YORK-PRESBYTERIAN/LOWER MANHATTAN HOSPITAL,Canal St 4-6-6 Express
2,50th St,1-2,"(40.76172799961419, -73.98384899986625)",0.588702,MOUNT SINAI WEST,50th St 1-2
3,Bergen St,2-3-4,"(40.68086213682956, -73.97499915116808)",0.686953,BROOKLYN HOSPITAL CENTER,Bergen St 2-3-4
4,Pennsylvania Ave,3-4,"(40.66471445143568, -73.89488591154061)",1.134439,BROOKDALE HOSPITAL MEDICAL CENTER,Pennsylvania Ave 3-4


The code needs to accomplish the following:
For each line in the turnstile dataframe,
Take the STATION_LINE value,
Look up the matching TurnstileID in the linker dataframe,
Take the corresponding LocatorID in the linker dataframe,
Look up the matching StatID in the hospital distance dataframe,
Return the corresponding Hospital Distance (mi) and place that in the turnstile dataframe.

Strategy: will be implemented as a function applied to the turnstile dataframe.
Function will take a string, the linker and hospital distance dataframes.
Will use the ".loc()" functions to retrieve the data at each step.
Convert TurnstileID to the linker's index and StatID to the hospital distance index.

In [10]:
linker_df.set_index('TurnstileID',inplace=True)
linker_df.head()

,LocatorIDs
TurnstileID,
103 ST-CORONA-7,103rd St - Corona Plaza 7
103 ST-1,103rd St 1
103 ST-6,103rd St 4-6-6 Express
103 ST-BC,103rd St A-B-C
104 ST-A,104th St A-S


In [11]:
hosp_dist_df.set_index('StatID',inplace=True)
hosp_dist_df.head()

,Station Name,Lines,Lat / Long,Hospital Distance (mi),Hospital ID
StatID,,,,,
Astor Pl 4-6-6 Express,Astor Pl,4-6-6 Express,"(40.73005400028978, -73.99106999861966)",0.367354,NEW YORK EYE AND EAR INFIRMARY
Canal St 4-6-6 Express,Canal St,4-6-6 Express,"(40.71880300107709, -74.00019299927328)",0.631470,NEW YORK-PRESBYTERIAN/LOWER MANHATTAN HOSPITAL
50th St 1-2,50th St,1-2,"(40.76172799961419, -73.98384899986625)",0.588702,MOUNT SINAI WEST
Bergen St 2-3-4,Bergen St,2-3-4,"(40.68086213682956, -73.97499915116808)",0.686953,BROOKLYN HOSPITAL CENTER
Pennsylvania Ave 3-4,Pennsylvania Ave,3-4,"(40.66471445143568, -73.89488591154061)",1.134439,BROOKDALE HOSPITAL MEDICAL CENTER


In [14]:
def linkup(stat_string,link_df,hosp_df):
    # must check for name in the linker lookup table
    # if not, pass None for this distance
    if stat_string in link_df['LocatorIDs']:
        locator = link_df.loc[stat_string,'LocatorIDs']
    else:
        return None
    dist = hosp_df.loc[locator,'Hospital Distance (mi)']
    return dist

In [15]:
# name the new column according to Natalie's convention
turnstile_df['MIN_HOSP_DIST'] = \
    turnstile_df.STATION_LINE.apply(linkup,link_df=linker_df,hosp_df=hosp_dist_df)


In [16]:
turnstile_df.sort_values('DAILY_ENTRIES',ascending=False)

,Unnamed: 0,STATION_LINE,DAILY_ENTRIES,MIN_HOSP_DIST
0,433,TIMES SQ 42 ST-1237ACENQRSW,156678.0,1.002665
1,88,34 ST-PENN STA-123ACE,138335.0,1.241920
2,299,GRD CNTRL-42 ST-4567S,120732.0,0.689167
3,86,34 ST-HERALD SQ-BDFMNQRW,106933.0,0.901751
4,26,14 ST-UNION SQ-456LNQRW,84564.0,0.346741
...,...,...,...,...
457,412,ROCKAWAY PARK B-AS,1067.0,4.485908
458,434,TOMPKINSVILLE-1,975.0,NaN
459,194,BEACH 98 ST-AS,695.0,3.640599
460,210,BROAD CHANNEL-AS,480.0,3.343090


In [19]:
del turnstile_df['Unnamed: 0']
turnstile_df.sort_values('MIN_HOSP_DIST',ascending=False).head(60)


,STATION_LINE,DAILY_ENTRIES,MIN_HOSP_DIST
457,ROCKAWAY PARK B-AS,1067.0,4.485908
461,BEACH 105 ST-AS,331.0,4.037159
459,BEACH 98 ST-AS,695.0,3.640599
460,BROAD CHANNEL-AS,480.0,3.343090
453,BEACH 90 ST-AS,1493.0,3.245013
316,GRANT AV-A,4621.0,2.913416
289,HOWARD BCH JFK-A,5240.0,2.834110
372,75 ST-ELDERTS-JZ,3387.0,2.730478
356,CRESCENT ST-JZ,3745.0,2.713364
373,80 ST-A,3386.0,2.668027


In [20]:
turnstile_df.sort_values('MIN_HOSP_DIST',ascending=False).to_csv('distance_descending.csv')